In [31]:
import numpy as np
import pandas as pd
import pathlib
import yaml
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import product

%matplotlib inline

In [2]:
np.random.seed(87654321)
CURR_PATH = pathlib.Path().parent

In [7]:
def read_txt_pd(exp: str = "FASERv2FCC_wide", otype: str = "inclusive", charge: str = "nu") -> pd.DataFrame:
    
    path = f"clipped_nan_binned_sysevents_{exp}_{otype}_{charge}.txt"
    fpath = CURR_PATH.joinpath(f"stat_syst_uncertainties/{path}")
    
    colnames = [
        "x_lower", 
        "x_upper", 
        "x_avg", 
        "Q2_lower", 
        "Q2_upper", 
        "Q2_avg", 
        "E_nu_lower", 
        "E_nu_upper", 
        "E_nu_avg", 
        "d^sigma/dxdQ2", 
        "N_events", 
        "N_events_errs", 
        "N_sys_errs", 
        "Percent_error_theta", 
        "Percent_error_Elepton", 
        "Percent_error_Ehadron", 
        "MC_Samples",
    ]
    
    return pd.read_csv(fpath, skiprows=2, delim_whitespace=True, names=colnames)

In [10]:
predictions = read_txt_pd()
predictions.head()

,x_lower,x_upper,x_avg,Q2_lower,Q2_upper,Q2_avg,E_nu_lower,E_nu_upper,E_nu_avg,d^sigma/dxdQ2,N_events,N_events_errs,N_sys_errs,Percent_error_theta,Percent_error_Elepton,Percent_error_Ehadron,MC_Samples
0,0.000189,0.000259,0.000230,4.0,10.0,5.390570,11.33635,40000.0,15037.280,68.08121,1914.486,43.75484,0.0,0.0,0.0,0.0,58178.0
1,0.000356,0.000489,0.000435,4.0,10.0,6.257304,11.33635,40000.0,12771.790,39.49984,10880.890,104.31150,0.0,0.0,0.0,0.0,77752.0
2,0.000489,0.000672,0.000557,4.0,10.0,5.457032,11.33635,40000.0,8705.647,28.58960,21806.780,147.67120,0.0,0.0,0.0,0.0,83840.0
3,0.000489,0.000672,0.000634,10.0,100.0,14.695920,11.33635,40000.0,14379.890,29.13603,6594.826,81.20854,0.0,0.0,0.0,0.0,14409.0
4,0.000672,0.000924,0.000804,4.0,10.0,6.082763,11.33635,40000.0,8678.035,21.33828,40198.550,200.49580,0.0,0.0,0.0,0.0,88220.0


In [11]:
MAP_ERROR_LABEL = {
    "Percent_error_Elepton": "El",
    "Percent_error_theta": "theta",
    "Percent_error_Ehadron": "Eh",
    "Percent_error_combined": "comb",
}

def load_input(
    exp: str = "FASERv2FCC_wide",
    otype: str = "inclusive",
    charge: str = "nu",
    pdfname: str = "NNPDF40_nnlo_as_01180",
    error: str = "Percent_error_Elepton",
) -> dict:
    # Read and Parse the central values
    partial_dataname = f"{exp}_{otype}_{charge}"
    data_name = f"diffxsec-{partial_dataname}-a1_{pdfname}"
    path_cv = CURR_PATH.joinpath(f"pineappl_tables/{data_name}.txt")
    
    if charge == "nu" or charge == "nub":
        # Extract the y & central value from pineappl tables
        column = 3 if charge == "nu" else 4 # Select projectile
        x_avg, y_avg, sigma = np.loadtxt(
            pathlib.Path(path_cv),
            usecols=(0, 1, column),
            unpack=True,
            skiprows=1,
        )
    elif charge == "nochargediscrimination":
        x_avg, y_avg, sigma_nu, sigma_nub = np.loadtxt(
            pathlib.Path(path_cv),
            usecols=(0, 1, 3, 4),
            unpack=True,
            skiprows=1,
        )
        sigma = sigma_nu + sigma_nub
    else:
        raise ValueEror(f"{charge} is not valid!")
    
    df_predictions = read_txt_pd(exp=exp, otype=otype, charge=charge)
    
    # Compute the corresponding systematic errors
    if error == "Percent_error_Elepton":
        syst_error = sigma * df_predictions["Percent_error_Elepton"].to_numpy()
    elif error == "Percent_error_theta":
        syst_error = sigma * df_predictions["Percent_error_theta"].to_numpy()
    elif error == "Percent_error_Ehadron":
        syst_error = sigma * df_predictions["Percent_error_Ehadron"].to_numpy()
    elif error == "Percent_error_combined":
        syst_error_El = sigma * df_predictions["Percent_error_Elepton"].to_numpy()
        syst_error_Th = sigma * df_predictions["Percent_error_theta"].to_numpy()
        syst_error_Eh = sigma * df_predictions["Percent_error_Ehadron"].to_numpy()
        syst_error = np.sqrt(syst_error_El**2 + syst_error_Th**2 + syst_error_Eh**2)
    else:
        raise ValueError(f"{error} is not a recognized error type!")
    
    # Extract the statistical events error
    num_events_error = df_predictions["N_events_errs"]
    stat_error = 1.0 / num_events_error * sigma
    
    # Check that the two files have the same knots
    np.testing.assert_allclose(x_avg, df_predictions["x_avg"], rtol=5e-3)
    
    # Add the statistical and systematic in quadrature
    comb_error = np.sqrt(syst_error**2 + stat_error**2)
    
    return {
        "x_values": df_predictions["x_avg"].to_numpy(),
        "q2_values": df_predictions["Q2_avg"].to_numpy(),
        "y_values": y_avg,
        "stat_error": stat_error.to_numpy(),
        "syst_error": syst_error,
        "comb_error": comb_error.to_numpy(),
        "sigma": sigma,
        "dataset_name": f"{partial_dataname}_{MAP_ERROR_LABEL[error]}",
    }

In [12]:
def fluctuate_data(central: np.ndarray, comb_error: np.ndarray) -> np.ndarray:
    pseudodata = []

    for cv, err in zip(central, comb_error):
        positive = False

        while positive is not True:
            shifted_cv = np.random.normal(cv, err)

            if shifted_cv > 0:
                positive = True

        pseudodata.append(shifted_cv)


    return np.asarray(pseudodata)

In [13]:
def fluctuate_data_covmat(central: np.ndarray, covmat: np.ndarray) -> np.ndarray:
    cholesky = np.linalg.cholesky(covmat)
    positive = False

    while positive is not True:
        random_samples = np.random.randn(central.shape[0])
        shift_data = cholesky @ random_samples
        pseudodata = central + shift_data

        if np.all(pseudodata >= 0):
            positive = True
    
    return pseudodata

In [15]:
# Function to get the Covmat from array
get_covmat = lambda arr: np.diag(arr**2)

In [34]:
def dump_newcommondata(df: pd.DataFrame, folder_name: str):
    #  dump the data file
    data_central = []
    for i in range(len(df["G"])):
        data_central.append(float(df.loc[i, "G"]))

    data_central_yaml = {"data_central": data_central}
    with open(f"{folder_name}/data.yaml", "w") as file:
        yaml.dump(data_central_yaml, file, sort_keys=False)

    # Write kin file
    kin = []
    for i in range(len(df["G"])):
        kin_value = {
            "x": {
                "min": None,
                "mid": float(df.loc[i, "x"]),
                "max": None,
            },
            "Q2": {"min": None, "mid": float(df.loc[i, "Q2"]), "max": None},
            "y": {"min": None, "mid": float(df.loc[i, "y"]), "max": None},
        }
        kin.append(kin_value)

    kinematics_yaml = {"bins": kin}

    with open(f"{folder_name}/kinematics.yaml", "w") as file:
        yaml.dump(kinematics_yaml, file, sort_keys=False)

    # Write unc file
    error = []
    for i in range(len(df)):
        e = {
            "stat": float(df.loc[i, "stat"]),
            "sys": float(df.loc[i, "sys"]),
        }
        error.append(e)

    error_definition = {
        "stat": {
            "description": "statistical uncertainty",
            "treatment": "ADD",
            "type": "UNCORR",
        },
        "sys": {
            "description": "systematic uncertainty",
            "treatment": "ADD",
            "type": "UNCORR",
        },
    }

    uncertainties_yaml = {"definitions": error_definition, "bins": error}

    with open(f"{folder_name}/uncertainties.yaml", "w") as file:
        yaml.dump(uncertainties_yaml, file, sort_keys=False)

In [83]:
def dump_metadata(expname: str, df: pd.DataFrame):      
    metadata = {'setname': expname,
     'version': 1,
     'version_comment': 'Initial implementation',
     'iNSPIRE': {'url': ''},
     'hepdata': {'url': '', 'version': 1},
     'nnpdf_metadata': 
                {
                    'nnpdf31_process': 'DIS CC',
                    'experiment': 'FASERV2'
                },
     'implemented_observables': [{'observable_name': 'fluctuated'.upper(),
       'observable': {'description': 'FPF at FCC',
        'label': '$d\sigma / dxdQ^2$',
        'units': ''},
       'process_type': 'DIS',
       'ndata': len(df['G']),
       'tables': [0],
       'npoints': [0],
       'plotting': {'kinematics_override': 'dis_sqrt_scale',
        'dataset_label': 'FPF at FCC',
        'y_label': '$d\sigma / dxdQ^2$',
        'plot_x': 'Q2',
        'line_by': ['x'],
        'figure_by': ['k2bins5']},
       'kinematic_coverage': ['x', 'Q2', 'y'],
       'kinematics': {'variables': {'x': {'description': 'momentum fraction',
          'label': '$x$',
          'units': ''},
         'Q2': {'description': 'virtuality', 'label': '$Q^2$', 'units': '$GeV^2$'},
         'y': {'description': 'inelasticity', 'label': '$y$', 'units': ''}},
        'file': 'kinematics.yaml'},
       'data_central': 'data.yaml',
       'data_uncertainties': ['uncertainties.yaml'],
       'theory': {
           'FK_tables': [[expname]],
        'operation': 'null'}}]
    }

    with open(f"fluctuated_data/{expname}/metadata.yaml", "w") as file:
        yaml.dump(metadata, file, sort_keys=False)

In [84]:
MAP_DATASET_NAMES = {
    "FASERv2FCC_wide_inclusive_nu_comb": 'FASERV2NU_FCC_WIDE_INCLUSIVE',
    "FASERv2FCC_wide_inclusive_nub_comb": 'FASERV2NB_FCC_WIDE_INCLUSIVE',
    "FASERv2FCC_inclusive_nu_comb": 'FASERV2NU_FCC_INCLUSIVE',
    "FASERv2FCC_inclusive_nub_comb": 'FASERV2NB_FCC_INCLUSIVE',
    "FASERv2FCC_deep_inclusive_nu_comb": 'FASERV2NU_FCC_DEEP_INCLUSIVE',
    "FASERv2FCC_deep_inclusive_nub_comb": 'FASERV2NB_FCC_DEEP_INCLUSIVE',
}

def dump_fluctuated_data(
    exps: list = [
        "FASERv2FCC_wide",
        "FASERv2FCC",
        "FASERv2FCC_deep",
    ],
    processes: list = ["inclusive"],
    charges: list = [
        "nu",
        "nub",
        # "nochargediscrimination"
    ],
    error: list = [
        "Percent_error_combined",
        # "Percent_error_Elepton",
        # "Percent_error_Ehadron",
        # "Percent_error_theta",
    ],
):
    for exp, proc, charge, err in product(exps, processes, charges, error):
        if exp == "FASERv" and proc == "charm":
            pass
        elif exp == "AdvSND" and charge == "nub":
            pass
        else:
            print(f"Fluctuating '{exp}' - '{proc}' - '{charge}' - '{err}'")
            load_results = load_input(exp=exp, otype=proc, charge=charge, error=err)
            
            # Compute the covariance matrix using the combined error
            # covmat = get_covmat(load_results["comb_error"])
    
            # Fluctuate the central values
            fluctuated_sigma = fluctuate_data(
                central=load_results["sigma"],
                comb_error=load_results["comb_error"],
            )
            
            # Combine everything into an array
            fluctuated_predictions = [
                load_results["x_values"],
                load_results["y_values"],
                load_results["q2_values"],
                fluctuated_sigma,
                load_results["stat_error"],
                load_results["syst_error"],
            ]

            fluctuated_predictions_df = {
                "x": load_results["x_values"],
                "y": load_results["y_values"],
                "Q2": load_results["q2_values"],
                "G": fluctuated_sigma,
                "stat": load_results["stat_error"],
                "sys": load_results["syst_error"],
            }
            fluctuated_predictions_df = pd.DataFrame(fluctuated_predictions_df)
            
            # Dump the final results
            # filename = f"{load_results['dataset_name']}_fluctuated"
            # save_path = CURR_PATH.joinpath(f"fluctuated_data/{filename}.txt")
            # np.savetxt(save_path, np.column_stack(fluctuated_predictions))
            datasetname = MAP_DATASET_NAMES[load_results['dataset_name']]
            foldername = f"fluctuated_data/{datasetname}"
            pathlib.Path(foldername).mkdir(exist_ok=True)
            dump_newcommondata(fluctuated_predictions_df, folder_name=foldername)
            dump_metadata(datasetname, fluctuated_predictions_df)

In [85]:
dump_fluctuated_data()

Fluctuating 'FASERv2FCC_wide' - 'inclusive' - 'nu' - 'Percent_error_combined'
Fluctuating 'FASERv2FCC_wide' - 'inclusive' - 'nub' - 'Percent_error_combined'
Fluctuating 'FASERv2FCC' - 'inclusive' - 'nu' - 'Percent_error_combined'
Fluctuating 'FASERv2FCC' - 'inclusive' - 'nub' - 'Percent_error_combined'
Fluctuating 'FASERv2FCC_deep' - 'inclusive' - 'nu' - 'Percent_error_combined'
Fluctuating 'FASERv2FCC_deep' - 'inclusive' - 'nub' - 'Percent_error_combined'
